In [ ]:
import torch

In [ ]:
%env CUDA_VISIBLE_DEVICES=0
torch.cuda.is_available()

In [ ]:
import scvi

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import hotspot
import pickle
import seaborn as sns


## load Boiarsky et al MM data

In [ ]:
!gsutil cp gs://rebecca-summer23/cd138_adata_MOREHVG_noIG.h5ad /tmp

In [ ]:
cd138_adata = sc.read_h5ad("/tmp/cd138_adata_MOREHVG_noIG.h5ad")

In [ ]:
sc.pl.umap(cd138_adata, color=["logW20","logW24"])

# compute UMAP on the basis of the 15 NMF modules

In [ ]:
cd138_adata.obsm['X_nmf'] = cd138_adata.obs.loc[:,['W'+str(i) for i in [3,4,5,8,9,11,16,20,24,28]]]

In [ ]:
sc.pp.neighbors(cd138_adata, use_rep="X_nmf")
sc.tl.umap(cd138_adata)

In [ ]:
sc.pl.umap(cd138_adata, color=["person","disease_stage"])

In [ ]:
sc.pl.umap(cd138_adata, color=['logW'+str(i) for i in [3,4,5,8,9,11,16,20,24,28]], ncols=2, size=12)

# compute UMAP on the basis of all NMF modules

In [ ]:
cd138_adata.obsm['X_nmf'] = cd138_adata.obs.loc[:,['W'+str(i) for i in np.arange(28)+1]]

In [ ]:
sc.pp.neighbors(cd138_adata, use_rep="X_nmf")
sc.tl.umap(cd138_adata)

In [ ]:
sc.pl.umap(cd138_adata, color=["person","disease_stage"])

In [ ]:
sc.pl.umap(cd138_adata, color=['logW'+str(i) for i in [3,4,5,8,9,11,16,20,24,28]], ncols=2, size=12)

## get CNV info (inferCNV), annotate genes that are in amplified or deleted regions

### write necessary input files to infercnv to disk (gene pos file created in R)

#### counts file

In [ ]:
pd.DataFrame(cd138_adata.layers['counts'].todense(), index = cd138_adata.obs.index, columns = cd138_adata.var.index).T.rename_axis(None, axis=1).to_csv("data/infercnv_input_files/cd138_mm/counts.txt", sep="\t")

#### sample annotation file

In [ ]:
pd.DataFrame(cd138_adata[~pd.isna(cd138_adata.obs.ground_truth)].obs['ground_truth'].astype("str") +"_" + cd138_adata[~pd.isna(cd138_adata.obs.ground_truth)].obs['person'].astype("str")).reset_index().to_csv("data/infercnv_input_files/cd138_mm/cell_annots.txt", sep="\t", header=False, index=False)


In [ ]:
#get list of normals to pass into infercnv function in R 
normals = list(pd.Series(cd138_adata[~pd.isna(cd138_adata.obs.ground_truth)].obs['ground_truth'].astype("str") +"_" + cd138_adata[~pd.isna(cd138_adata.obs.ground_truth)].obs['person'].astype("str")).drop_duplicates())
normals = [n for n in normals if "healthy" in n]
normals


In [ ]:
#ncells = 1000 
pt_list = ["MM-1","MM-2","SMM-3","NBM-10","SMM-8"]

#make a small version of file for infercnv with ncells
small_counts = pd.DataFrame(cd138_adata[cd138_adata.obs.person.isin(pt_list)].layers['counts'].todense(), index = cd138_adata.obs.index[cd138_adata.obs.person.isin(pt_list)], columns = cd138_adata.var.index).T.rename_axis(None, axis=1)
small_counts.to_csv("data/infercnv_input_files/cd138_mm/small_counts.txt", sep="\t")

In [ ]:
#small sample annots
small_cell_annots = pd.DataFrame(cd138_adata[cd138_adata.obs.index.isin(small_counts.columns)].obs['ground_truth'].astype("str") +"_" + cd138_adata[cd138_adata.obs.index.isin(small_counts.columns)].obs['person'].astype("str")).reset_index()
small_cell_annots.to_csv("data/infercnv_input_files/cd138_mm/small_cell_annots.txt", sep="\t", header=False, index=False)

In [ ]:
#get list of normals to pass into infercnv function in R 
small_normals = list(pd.Series(cd138_adata[cd138_adata.obs.index.isin(small_counts.columns)].obs['ground_truth'].astype("str") +"_" + cd138_adata[cd138_adata.obs.index.isin(small_counts.columns)].obs['person'].astype("str")).drop_duplicates())
small_normals = [n for n in small_normals if "healthy" in n]
small_normals


In [ ]:
small_cell_annots.iloc[:,1].value_counts()

In [ ]:
len(small_cell_annots)

## run vanilla scVI 

In [ ]:
scvi.model.SCVI.setup_anndata(cd138_adata, layer="counts")

In [ ]:
vae = scvi.model.SCVI(cd138_adata, gene_likelihood='nb')

In [ ]:
vae.train()

In [ ]:
#pickle scvi results

# open a file, where you ant to store the data
file = open('outputs/vanilla_scvi_cd138.pkl', 'wb')

# dump information to that file
pickle.dump(vae, file)

# close the file
file.close()


In [ ]:
!gsutil cp gs://rebecca-summer23/outputs/vanilla_scvi_cd138.pkl /tmp

In [ ]:
# read pickled scvi results

file = open('/tmp/vanilla_scvi_cd138.pkl', 'rb')

# dump information to that file
vae = pickle.load(file)

# close the file
file.close()

In [ ]:
cd138_adata.obsm["X_scvi"] = vae.get_latent_representation()

In [ ]:
#save original UMAP
cd138_adata.obsm["X_umap_pca-based"] = cd138_adata.obsm['X_umap']

In [ ]:
sc.pp.neighbors(cd138_adata, use_rep="X_scvi")
sc.tl.umap(cd138_adata)
sc.pl.umap(cd138_adata, color="person")

In [ ]:
sc.pl.umap(cd138_adata, color=["ground_truth", "disease_stage", "prolif_idx"])

In [ ]:
sc.pl.embedding(cd138_adata, basis="X_umap_pca-based", color=["ground_truth", "disease_stage", "prolif_idx"])

#### differential expression

In [ ]:
scvi_de = vae.differential_expression(adata=cd138_adata, groupby="person", group1=None, idx2=cd138_adata.obs.disease_stage=="NBM", 
                                      mode='change', delta=0.25, batch_size=None, all_stats=True, batch_correction=False, batchid1=None, batchid2=None, fdr_target=0.05, silent=False, weights='uniform', filter_outlier_cells=False)

In [ ]:
scvi_de = scvi_de[scvi_de['is_de_fdr_0.05']]

In [ ]:
scvi_de[scvi_de.group1=="MM-8"]

## inspect latent space: hotspot analysis

In [ ]:
#filter to hvg
cd138_adata_hvg = cd138_adata[:,cd138_adata.var.highly_variable]
cd138_adata_hvg.layers['counts'] = cd138_adata.layers['counts'][:,cd138_adata.var.highly_variable]

In [ ]:
hs = hotspot.Hotspot(
    cd138_adata_hvg,
    layer_key="counts",
    model='danb',
    latent_obsm_key="X_scvi"
)

In [ ]:
hs.create_knn_graph(weighted_graph=True, n_neighbors=50)

In [ ]:
hs_results = hs.compute_autocorrelations()

In [ ]:
#pickle hotspot results

# open a file, where you ant to store the data
file = open('outputs/hotspot_vanilla_scvi_cd138.pkl', 'wb')

# dump information to that file
pickle.dump(hs, file)

# close the file
file.close()


In [ ]:
# read pickled hotspot results

file = open('outputs/hotspot_vanilla_scvi_cd138.pkl', 'rb')

# dump information to that file
hs = pickle.load(file)

# close the file
file.close()

In [ ]:
hs_genes = hs.results.loc[hs.results.FDR < 0.05].index # Select genes #1e-323 seems to be some hardcoded floor on the FDR, if move threshold lower than that there are 0 genes
len(hs_genes)

In [ ]:
local_correlations = hs.compute_local_correlations(hs_genes, jobs=12) # jobs for parallelization

In [ ]:
#pickle hotspot results

# open a file, where you ant to store the data
file = open('outputs/hotspot_vanilla_scvi_cd138.pkl', 'wb')

# dump information to that file
pickle.dump(hs, file)

# close the file
file.close()


In [ ]:
# read pickled hotspot results

file = open('outputs/hotspot_vanilla_scvi_cd138.pkl', 'rb')

# dump information to that file
hs = pickle.load(file)

# close the file
file.close()

In [ ]:
modules = hs.create_modules(min_gene_threshold=30, core_only=True, fdr_threshold=0.05)

In [ ]:
hs.plot_local_correlations()

In [ ]:
module_scores = hs.calculate_module_scores()

In [ ]:
#plot module score on UMAP
cd138_adata.obsm["hotspot_module_scores"] = module_scores

In [ ]:
module_scores.columns = ["hs_mod_"+str(c) for c in module_scores.columns]
module_scores.head()

In [ ]:
cd138_adata.obs = cd138_adata.obs.merge(module_scores, left_index=True, right_index=True)

In [ ]:
#UMAP BASED ON PCA (orig from paper)
sc.pl.embedding(cd138_adata, basis="X_umap_pca-based", color=module_scores.columns)

In [ ]:
#UMAP BASED ON scvi
sc.pl.umap(cd138_adata, color=module_scores.columns)

### quantify extent of overlap b/w hotspot genes and CNV genes

In [ ]:
gtf = pd.read_table("data/infercnv_input_files/cd138_mm/gene_pos.txt", header=None)
gtf.columns = ["gene","chrom","start","end"]

In [ ]:
gtf

In [ ]:
#UMAP BASED ON PCA (orig from paper)
sc.pl.umap(cd138_adata, color="person", legend_loc="on data")

In [ ]:
#UMAP BASED ON PCA (orig from paper)
sc.pl.umap(cd138_adata, color="person")

### SMM7 is mod 9, acc to FISH and inferCNV has monosomy 13 -- which chromosomes are hotspot module 9 genes from? (histogram)

In [ ]:
mod=9

res = pd.DataFrame(gtf[gtf.gene.isin(hs.modules[hs.modules==mod].index)].chrom.value_counts().reset_index()) # no enrichment for chromosome 13
res.chrom = pd.Categorical(res.chrom, categories=[str(i) for i in np.arange(22)+1] + ["X", "Y"], ordered=True)
sns.barplot(data=res.sort_values("chrom"), x="chrom", y="count")

### MM1 is mod 4, acc to FISH has  acc to FISH has t(4;14), 1q duplication, monosomy 13.; inferCNV has lots of other stuff going on too

In [ ]:
mod=4

res = pd.DataFrame(gtf[gtf.gene.isin(hs.modules[hs.modules==mod].index)].chrom.value_counts().reset_index()) # no enrichment for chromosome 13
res.chrom = pd.Categorical(res.chrom, categories=[str(i) for i in np.arange(22)+1] + ["X", "Y"], ordered=True)
sns.barplot(data=res.sort_values("chrom"), x="chrom", y="count")

### MM2 is mod 6, acc to FISH has  acc to FISH has tetrasomy 9, trisomy 11 and 15.; inferCNV has lots of other stuff going on too

In [ ]:
mod=6

res = pd.DataFrame(gtf[gtf.gene.isin(hs.modules[hs.modules==mod].index)].chrom.value_counts().reset_index()) # no enrichment for chromosome 13
res.chrom = pd.Categorical(res.chrom, categories=[str(i) for i in np.arange(22)+1] + ["X", "Y"], ordered=True)
sns.barplot(data=res.sort_values("chrom"), x="chrom", y="count")

# Would we have recovered more CNV genes if we didn't filter to HVG? Too expensive to run hotspot will all genes, but can use rank_genes_groups

In [ ]:
sc.tl.rank_genes_groups(cd138_adata, groupby='leiden', method='wilcoxon')

In [ ]:
leiden_genes_df = sc.get.rank_genes_groups_df(cd138_adata, group=None, key='rank_genes_groups_filtered')

In [ ]:
sc.pl.umap(cd138_adata, color=["leiden", "person"], ncols=1, legend_loc = "on data", legend_fontoutline=2)

In [ ]:
#only returns upregulated genes
leiden_genes_df.head()

### SMM7 is clust 5, acc to FISH and inferCNV has monosomy 13

In [ ]:
clust="5"
res = pd.DataFrame(gtf[gtf.gene.isin(leiden_genes_df[leiden_genes_df.group==clust].names)].chrom.value_counts().reset_index()) # no enrichment for chromosome 1, 13
res.chrom = pd.Categorical(res.chrom, categories=[str(i) for i in np.arange(22)+1] + ["X", "Y"], ordered=True)

In [ ]:
sns.barplot(data=res.sort_values("chrom"), x="chrom", y="count")

### MM1 is leiden 2, acc to FISH has  acc to FISH has t(4;14), 1q duplication, monosomy 13.; inferCNV has lots of other stuff going on too

In [ ]:
clust="2"
res = pd.DataFrame(gtf[gtf.gene.isin(leiden_genes_df[leiden_genes_df.group==clust].names)].chrom.value_counts().reset_index()) # no enrichment for chromosome 1, 13
res.chrom = pd.Categorical(res.chrom, categories=[str(i) for i in np.arange(22)+1] + ["X", "Y"], ordered=True)
sns.barplot(data=res.sort_values("chrom"), x="chrom", y="count")

### MM2 is clust 0, acc to FISH has tetrasomy 9, trisomy 11 and 15.; inferCNV has lots of other stuff going on too

In [ ]:
clust="0"
res = pd.DataFrame(gtf[gtf.gene.isin(leiden_genes_df[leiden_genes_df.group==clust].names)].chrom.value_counts().reset_index()) # no enrichment for chromosome 1, 13
res.chrom = pd.Categorical(res.chrom, categories=[str(i) for i in np.arange(22)+1] + ["X", "Y"], ordered=True)
sns.barplot(data=res.sort_values("chrom"), x="chrom", y="count")

# Create gene set scores for CNV genes

In [ ]:
cnv_genes = pd.read_table("outputs/infercnv/cd138/HMM_CNV_predictions.HMMi6.hmm_mode-samples.Pnorm_0.5.pred_cnv_genes.dat", sep="\t")

In [ ]:
# parse sample name
cnv_genes['sid'] = [s.split(".")[0] for s in cnv_genes.cell_group_name]
cnv_genes['sid'] = [s.split("_")[1] for s in cnv_genes.sid]

In [ ]:
cnv_genes

In [ ]:
# only want to include upregulated genes in gene score
upgenes = cnv_genes[cnv_genes.state.isin([4,5,6])]

In [ ]:
sample_cnv_upgenes = [list(upgenes[upgenes.sid==sid].gene) for sid in upgenes.sid]

In [ ]:
sample_cnv_upgenes = dict(zip([sid for sid in upgenes.sid], sample_cnv_upgenes))

In [ ]:
[len(sample_cnv_upgenes[i]) for i in sample_cnv_upgenes.keys()]

In [ ]:
#score adata for each genesets
[sc.tl.score_genes(cd138_adata, sample_cnv_upgenes[geneset], ctrl_size=len(sample_cnv_upgenes[geneset]), score_name='score_'+geneset+"_cnv_upgenes") 
 for geneset in sample_cnv_upgenes.keys()]


In [ ]:
cd138_adata

In [ ]:
sc.pl.umap(cd138_adata, color=['score_MGUS-2_cnv_upgenes', 'score_MGUS-3_cnv_upgenes', 'score_MGUS-6_cnv_upgenes', 'score_MM-1_cnv_upgenes', 'score_MM-2_cnv_upgenes', 'score_MM-3_cnv_upgenes', 'score_MM-4_cnv_upgenes', 'score_MM-5_cnv_upgenes', 'score_MM-6_cnv_upgenes', 'score_MM-7_cnv_upgenes', 'score_MM-8_cnv_upgenes', 'score_SMM-1_cnv_upgenes', 'score_SMM-10_cnv_upgenes', 'score_SMM-11_cnv_upgenes', 'score_SMM-12_cnv_upgenes', 'score_SMM-2_cnv_upgenes', 'score_SMM-3_cnv_upgenes', 'score_SMM-4_cnv_upgenes', 'score_SMM-5_cnv_upgenes', 'score_SMM-7_cnv_upgenes', 'score_SMM-8_cnv_upgenes'])

In [ ]:
sc.pl.umap(cd138_adata, color=["person"], ncols=1, legend_loc = "on data", legend_fontoutline=2)

In [ ]:
sc.pl.umap(cd138_adata, color=['logW3', 'logW4', 'logW5', 'logW8', 'logW9', 'logW11', 'logW16', 'logW20', 'logW24', 'logW28'])

## Do we have translocation data recorded?

In [ ]:
cd138_adata.obs[[ 'tx',
 'HRD',
 'driver event',
 'driver_event_specific']]

In [ ]:
sc.pl.umap(cd138_adata, color=['driver event',
 'driver_event_specific'])

In [ ]:
cd138_adata.var.highly_variable.sum()